# 📈 Analyse Long-terme des Recettes

## Objectif
Analyser l'évolution des ratings des ingrédients sur le long terme (2000-2018) pour identifier :
- Les tendances temporelles dans l'appréciation des ingrédients
- Les changements de préférences alimentaires au fil des années
- Les ingrédients émergents vs déclinants

## Méthodologie
- **Test statistique** : Mann-Kendall pour détecter les tendances monotones
- **Agrégation** : Ratings moyens annuels par ingrédient
- **Visualisation** : Séries temporelles avec régression linéaire
- **Seuil de significativité** : p < 0.05